In [6]:
import sys
from pathlib import Path
from datetime import datetime

# 1. Setup Path (Points to project root so we can import 'src')
# Adjust '.parent' if your notebook is deeper in subfolders
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.append(str(project_root / 'src'))

# 2. Import the Reader
from atspm.data import get_legacy_dataframe, convert_to_datetime

# 3. Define Parameters
# Update folder name to match your specific intersection
tz = 'US/Mountain'
db_path = project_root / 'intersections' / '201_SH-55_and_Banks-Lowman_Rd' / '201_data.db'
#db_path = project_root / 'intersections' / '304_US-20-26_Franklin_Rd_and_Middleton_Rd' / '304_data.db'

tz = 'US/Pacific'
db_path = project_root / 'intersections' / '2068_US-95_and_SH-8' / '2068_data.db'
#db_path = project_root / 'intersections' / '2070_US-95_and_Palouse_River_Dr' / '2070_data.db'


start_date = datetime(2026, 2, 12)
end_date = datetime(2026, 2, 13)

# 4. Fetch Data (Legacy "Flat" Format)
# Returns columns: [TS_start, Code, ID, Cycle_start, Coord_plan]
df = get_legacy_dataframe(db_path, start_date, end_date, timezone=tz)

# 5. Inspect
print(f"Loaded {len(df):,} rows.")
convert_to_datetime(df.head())

Loaded 157,542 rows.


c:\Users\rhansen\Documents\Python\pyatspm\src\atspm\data\reader.py:194: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  merged_df['cycle_start'].fillna(first_cycle, inplace=True)
c:\Users\rhansen\Documents\Python\pyatspm\src\atspm\data\reader.py:195: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series throug

,TS_start,Code,ID,Cycle_start,Coord_plan
0,2026-02-12 03:15:18.099999905-08:00,82,37,NaT,NaN
1,2026-02-12 03:15:22.700000048-08:00,81,37,NaT,NaN
2,2026-02-12 03:15:23.900000095-08:00,82,51,NaT,NaN
3,2026-02-12 03:15:24.700000048-08:00,81,51,NaT,NaN
4,2026-02-12 03:15:24.799999952-08:00,82,51,NaT,NaN


In [7]:
for p, events in df[df.Code.isin([1,8,9,10,11,12])].groupby('ID'):
    print(f"Phase {p}: {events.head(20)}")

Phase 4:                                TS_start  Code  ID  \
492 2026-02-12 04:10:10.099999905-08:00     1   4   
511 2026-02-12 04:10:20.599999905-08:00    12   4   
524 2026-02-12 04:10:27.299999952-08:00     1   4   
554 2026-02-12 04:10:54.299999952-08:00     8   4   
560    2026-02-12 04:10:57.500000-08:00     9   4   
561    2026-02-12 04:10:57.500000-08:00    10   4   
565 2026-02-12 04:11:00.700000048-08:00    11   4   
566 2026-02-12 04:11:00.700000048-08:00    12   4   
610           2026-02-12 04:11:50-08:00     1   4   
622           2026-02-12 04:11:55-08:00     8   4   
624 2026-02-12 04:11:58.200000048-08:00     9   4   
625 2026-02-12 04:11:58.200000048-08:00    10   4   
629 2026-02-12 04:12:01.400000095-08:00    11   4   
630 2026-02-12 04:12:01.400000095-08:00    12   4   
657 2026-02-12 04:12:23.400000095-08:00     1   4   
674 2026-02-12 04:12:50.400000095-08:00     8   4   
677 2026-02-12 04:12:53.599999905-08:00     9   4   
678 2026-02-12 04:12:53.599999905-08:

In [5]:
print(convert_to_datetime(df[df.Code<0]))

                                  TS_start  Code  ID  \
28914  2026-02-11 16:44:58.599999905-08:00    -1  -1   
217174 2026-02-13 09:14:59.199999809-08:00    -1  -1   
400902 2026-02-15 01:29:45.399999857-08:00    -1  -1   

                               Cycle_start  Coord_plan  
28914  2026-02-11 16:43:54.799999952-08:00         0.0  
217174           2026-02-13 09:14:25-08:00         1.0  
400902 2026-02-15 01:29:40.299999952-08:00         0.0  


In [8]:
import sqlite3
import pandas as pd
from pathlib import Path

# 1. Point to your database
# (Make sure this path is correct relative to where you run the notebook)
db_path = Path('../intersections/2068_US-95_and_SH-8/2068_data.db')
#db_path = Path('../intersections/201_SH-55_and_Banks-Lowman_Rd/201_data.db')

print(f"Connecting to: {db_path.resolve()}")

with sqlite3.connect(db_path) as conn:
    # A. Check what tables exist
    print("\n--- TABLES ---")
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
    print(tables)

    # B. Check Metadata (Is the Timezone correct?)
    print("\n--- METADATA ---")
    try:
        meta = pd.read_sql("SELECT * FROM metadata", conn)
        print(meta.T) # Transpose for readability
    except Exception as e:
        print(f"Could not read metadata: {e}")

    # C. Check Ingestion Log (What files actually got loaded?)
    print("\n--- INGESTION LOG (First 5) ---")
    try:
        log = pd.read_sql("SELECT * FROM ingestion_log ORDER BY span_start", conn)
        print(log)
    except Exception as e:
        print(f"Could not read ingestion_log: {e}")

    # D. Check Events Range (The most important part!)
    # This tells us the EXACT time range of your data in UTC Epoch
    print("\n--- EVENTS SUMMARY ---")
    try:
        query = """
        SELECT 
            MIN(timestamp) as min_ts, 
            MAX(timestamp) as max_ts, 
            COUNT(*) as total_rows 
        FROM events
        """
        stats = pd.read_sql(query, conn)
        
        # Convert to readable text for us humans
        if stats['total_rows'][0] > 0:
            stats['min_ts'] = pd.to_datetime(stats['min_ts'], unit='s')
            stats['max_ts'] = pd.to_datetime(stats['max_ts'], unit='s')
            print(stats)
        else:
            print("⚠️ EVENTS TABLE IS EMPTY!")
            
    except Exception as e:
        print(f"Could not read events: {e}")

    # E. Check Config
    print("\n--- CONFIG ---")
    try:
        cfg = pd.read_sql("SELECT * FROM config", conn)
        print(cfg.head())
    except Exception as e:
        print(f"Could not read config: {e}")

conn.close()

Connecting to: C:\Users\rhansen\Documents\Python\pyatspm\intersections\2068_US-95_and_SH-8\2068_data.db

--- TABLES ---
              name
0           events
1           config
2  sqlite_sequence
3         metadata
4    ingestion_log
5           cycles

--- METADATA ---
                            0
lock_id                     1
intersection_id          2068
intersection_name        None
controller_ip      10.37.2.68
detection_type      Evo Radar
detection_ip       10.37.2.86
major_road_route        US-95
major_road_name          None
minor_road_route         SH-8
minor_road_name          None
latitude                 None
longitude                None
timezone           US/Pacific
agency_id                None

--- INGESTION LOG (First 5) ---
     span_start      span_end                processed_at  row_count
0  1.770842e+09  1.770856e+09  2026-02-18T07:56:00.385744      28914
1  1.770895e+09  1.771002e+09  2026-02-18T07:56:00.385744     188259
2  1.771040e+09  1.771147e+09  2026-02-

In [13]:
with sqlite3.connect(db_path) as conn:
    cycles = pd.read_sql("SELECT * FROM cycles", conn)

In [14]:
cycles.head()

,cycle_start,coord_plan,detection_method
0,1.770842e+09,0,ring_barrier_config
1,1.770843e+09,0,ring_barrier_config
2,1.770843e+09,0,ring_barrier_config
3,1.770843e+09,0,ring_barrier_config
4,1.770843e+09,0,ring_barrier_config


In [12]:
log[(log.file_timestamp != log.file_timestamp.shift(-1)-(15*60))]

,filename,file_timestamp,processed_at,row_count
52,ECON_10.37.23.200_2025_09_19_1300.datZ,1.758308e+09,2026-02-17T07:22:26.682190,3963
756,ECON_10.37.23.200_2025_11_25_1300.datZ,1.764101e+09,2026-02-17T07:22:42.547608,1228
757,ECON_10.37.23.200_2025_11_25_1327.datZ,1.764102e+09,2026-02-17T07:22:42.547608,841
2839,ECON_10.37.23.200_2025_12_17_0545.datZ,1.765976e+09,2026-02-17T07:23:30.640350,605
2840,ECON_10.37.23.200_2025_12_17_0607.datZ,1.765977e+09,2026-02-17T07:23:30.640350,633
3736,ECON_10.37.23.200_2025_12_26_1400.datZ,1.766783e+09,2026-02-17T07:23:49.541268,1626
3737,ECON_10.37.23.200_2025_12_26_1428.datZ,1.766784e+09,2026-02-17T07:23:49.541268,587
7302,ECON_10.37.23.200_2026_02_01_1730.datZ,1.769992e+09,2026-02-17T07:27:21.535087,776
7303,ECON_10.37.23.200_2026_02_01_1754.datZ,1.769994e+09,2026-02-17T07:27:21.535087,914
7386,ECON_10.37.23.200_2026_02_02_1430.datZ,1.770068e+09,2026-02-17T07:27:23.277916,1425


In [5]:
conn.close()

NameError: name 'conn' is not defined

In [3]:
convert_to_datetime(df[df.Code==1])

,TS_start,Code,ID,Cycle_start,Coord_plan
164,2026-02-17 00:01:49.200000048-08:00,1,4,2026-02-16 23:58:41.700000048-08:00,0.0
194,2026-02-17 00:02:07.400000095-08:00,1,6,2026-02-16 23:58:41.700000048-08:00,0.0
220,2026-02-17 00:02:39.299999952-08:00,1,4,2026-02-17 00:02:39.299999952-08:00,0.0
243,2026-02-17 00:02:55.799999952-08:00,1,6,2026-02-17 00:02:39.299999952-08:00,0.0
284,2026-02-17 00:05:55.799999952-08:00,1,4,2026-02-17 00:05:55.799999952-08:00,0.0
...,...,...,...,...,...
70425,2026-02-17 12:42:03.500000-08:00,1,8,2026-02-17 12:40:41.799999952-08:00,0.0
70490,2026-02-17 12:42:25-08:00,1,4,2026-02-17 12:42:25-08:00,0.0
70590,2026-02-17 12:43:03.500000-08:00,1,6,2026-02-17 12:42:25-08:00,0.0
70700,2026-02-17 12:43:41.400000095-08:00,1,8,2026-02-17 12:42:25-08:00,0.0
